In [ ]:
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/jhoward/.fastai/data/imdb_sample')

In [ ]:
data_lm = (TextList.from_csv(path, 'texts.csv', create_func=None, col='text')
                   .random_split_by_pct()
                   .label_for_lm()
                   .preprocess()
                   .databunch())

In [ ]:
data_lm.show_batch()

idx,text
0,"xxfld 1 this movie is the first of the six infamous xxunk xxunk movies and is one of the best . at the same time , it looks realistic and unrealistic , just knowing that the movie is fake . the story is about a woman who got captured and is tortured in a lot of different ways . a man in the beginning of the story xxunk a letter without a return address , and it includes a manga video , showing the torture . the men who capture her are testing the limits"
1,"senses and xxunk his guilt willing to face the music and then , after his three year sentence is up , get his life back together . \n\n * * * xxup spoiler xxup xxunk * * * hearing rumors from fellow convicts that rose and his best friend fred were having an affair behind his back jimmy broke out of prison ending up a xxunk from the law . it 's at fred 's circus , where he works as both xxunk and barker , that jimmy in seeing that rose as well as"
2,"film has no victims , except lily 's childhood , which was destroyed by an abusive exploitative father . the destructive relationship with her father suggests lily 's hidden xxunk for using men to xxunk without regard for their fate . while lily is cynical and obvious in her approach , the men she xxunk xxunk xxunk wives and xxunk to trade jobs for sexual xxunk . perhaps the bank failures in the 1930 's xxunk less to xxunk than to morally corrupt executives xxunk by ambitious women . \n\n the plot moves fast ,"
3,", it is less than clear why would she burn phillip 's money ( xxunk intended different in his novel ) . after all , she could as well steal it and drunk xxunk of champagne . \n\n for modern standards the film is a bit outdated , but each subsequent time you watch it , you can reveal new interesting details due to superior acting , fascinating music and original editing , so it does deserve the highest possible mark . xxfld 1 i was surprised , that ' ' the secret fury ''"
4,"did get off slightly better than poor old bela lugosi but not by much . \n\n boris does his best and give him credit for trying to hold this mess together . the xxunk background music does n't help and xxunk from any xxunk moment . apart from boris , the rest of the mexican cast are dubbed into some strange , xxunk , english xxunk that is reminiscent of the type used in porn films of the late seventies . \n\n at a guess i think it 's edgar allen poe 's ' house"
5,", but felt that it would be unfair . there was a representation of a storm , but where in your mind do you xxunk a small boy xxunk over a toy boat ? it is symbolic of what ? i continued another ten minutes my finger xxunk over the ' off ' button , somewhere something would capture my interest . this is not shakespeare , it is not cinema . there is a time and place for it , but i will not waste my time and there is no place for it"
6,"peters , followed a long - standing tradition of xxunk who should be watched with the xxunk turned completely down . \n\n - the xxunk focus of the movie was a repeating , compound xxunk with xxunk xxunk . it never needed to be loaded and even had a xxunk when fired . it managed to shred the laws of physics , the integrity of the original legend , historical fact and plot suspense all by itself . \n\n - xxunk 's palace , xxunk , rather than being a xxunk long hall , apparently"
7,"somewhere a long way from a cinema . \n\n paul greengrass 's xxunk style worked exceptionally well on united xxunk which was a sentiment xxunk desperate to happen , but on bourne and his xxunk xxunk it just has the effect of xxunk the audience from involvement with the character . he runs . he jumps . he punches . he gets blown up . he xxunk tall buildings . yada yada yada . above all - he xxup survives . he survives like a plastic action man survives , which only makes the ridiculous"
8,"of the 1950s was justified to some xxunk - xxunk did not even exist yet . there were xxunk circumstances then . \n\n 4 out of 10 ( 1 point is given from the start , 1 point goes for the r

In [ ]:
data_clas = (TextList.from_csv(path, 'texts.csv', create_func=None, col='text')
                     .random_split_by_pct()
                     .label_from_df(cols=0)
                     .preprocess()
                     .databunch())

In [ ]:
data_clas.show_batch()

text,label
"xxfld 1 raising victor vargas : a review \n\n you know , raising victor vargas is like sticking your hands into a big , xxunk bowl of xxunk . it 's warm and gooey , but you 're not sure if it feels right . try as i might ,",negative
"xxfld 1 xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words",positive
"xxfld 1 now that che(2008 ) has finished its relatively short australian cinema run ( extremely limited xxunk screen in xxunk , after xxunk ) , i can xxunk join both xxunk of "" at the movies "" in taking steven soderbergh to task . \n\n it 's usually satisfying",negative
"xxfld 1 many xxunk that this is n't just a classic due to the fact that it 's the first 3d game , or even the first xxunk - up . it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly",positive
"xxfld 1 i really wanted to love this show . i truly , honestly did . \n\n for the first time , gay viewers get their own version of the "" the bachelor "" . with the help of his obligatory "" hag "" xxunk , james , a good",negative


In [ ]:
x,y = next(iter(data_clas.train_dl));
x

tensor([[  42,    1,    1,  ...,    1,    1,    1],
        [  40,    1,    1,  ...,    1,    1,    1],
        [2462,    1,    1,  ...,    1,    1,    1],
        ...,
        [   0,  886,  118,  ...,  310,    2,   35],
        [   4,   96,  164,  ...,   54,  146,  143],
        [  19,    4,   26,  ...,    4,   50,    4]], device='cuda:0')

In [ ]:
learn = language_model_learner(data_lm)

In [ ]:
learn = text_classifier_learner(data_clas)

In [ ]:
def open_text(fn:PathOrStr):
    with open(fn,'r') as f: return ''.join(f.readlines())

In [ ]:
def _treat_html(o:str)->str:
    return o.replace('\n','\\n')

def _text2html_table(items:Collection[Collection[str]], widths:Collection[int])->str:
    html_code = f"<table>"
    for w in widths: html_code += f"  <col width='{w}%'>"
    for line in items:
        html_code += "  <tr>\n"
        html_code += "\n".join([f"    <th>{_treat_html(o)}</th>" for o in line if len(o) >= 1])
        html_code += "\n  </tr>\n"
    return html_code + "</table>\n"

In [ ]:
class Text(ItemBase):
    def __init__(self, ids, text): self.data,self.text = ids,text
    def __str__(self):  return str(self.text)
    
    def show_batch(self, idxs:Collection[int], rows:int, ds:Dataset, figsize:Tuple[int,int]=(9,10))->None:
        from IPython.display import clear_output, display, HTML
        items = [['text', 'label']]
        for i in idxs[:rows]:
            x,y = ds[i]
            items.append([str(x), str(y)])
        display(HTML(_text2html_table(items, [90,10])))

class NumericalizedTextList(ItemList):
    def __init__(self, items:Iterator, vocab:Vocab=None, **kwargs):
        super().__init__(items, **kwargs)
        self.vocab = vocab
        
    def new(self, items:Iterator, **kwargs)->'NumericalizedTextList':
        return super().new(items=items, vocab=self.vocab, **kwargs)
    
    def get(self, i):
        o = super().get(i)
        return Text(o, self.vocab.textify(o))
    
class TextList(NumericalizedTextList):
    def __post_init(self): pass

class TextFilesList(TextList):
    def __init__(self, items:Iterator, create_func:Callable=None, path:PathOrStr='.'):
        texts = [open_text(fn) for fn in items]
        super().__init__(texts, create_func, path)

In [ ]:
data_lm = (TextList.from_csv(path, 'texts.csv', create_func=None, col='text')
                   .random_split_by_pct()
                   .label_for_lm()
                   .preprocess()
                   .databunch())

TypeError: label_for_lm() got an unexpected keyword argument 'template'

In [ ]:
class TextProcessor(PreProcessor):
    def __init__(self, tokenizer:Tokenizer=None, chunksize:int=10000,
                 vocab:Vocab=None, max_vocab:int=60000, min_freq:int=2):
        self.chunksize,self.max_vocab,self.min_freq = chunksize,max_vocab,min_freq
        self.tokenizer,self.vocab = ifnone(tokenizer, Tokenizer()),vocab

    def process(self, ds):
        tokens = []
        for i in progress_bar(range(0,len(ds),self.chunksize), leave=False):
            tokens += self.tokenizer.process_all(ds.items[i:i+self.chunksize])
        ds.items = tokens
        if self.vocab is None: self.vocab = Vocab.create(ds.items, self.max_vocab, self.min_freq)
        ds.vocab = self.vocab
        ds.items = np.array([self.vocab.numericalize(t) for t in ds.items])

In [ ]:
processor=TextProcessor()

In [ ]:
il = (TextList.from_csv(path, 'texts.csv', create_func=None, col='text', processor=processor)
        .random_split_by_pct()
        .label_from_df(cols=0)
     )

In [ ]:
len(il.valid.vocab.itos), len(il.train.vocab.itos)

(5913, 5913)

In [ ]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
il.add_test(df['text'].values);

In [ ]:
class C(int):
    

In [ ]:
il.train.y.classes,il.valid.y.classes,il.test.y.classes

(['negative', 'positive'], ['negative', 'positive'], ['negative', 'positive'])

In [ ]:
len(il.valid.vocab.itos), len(il.train.vocab.itos), len(il.test.vocab.itos)

(5913, 5913, 5913)

In [ ]:
data = il.databunch()

In [ ]:
data.show_batch(ds_type=DatasetType.Test)

text,label
"un - xxunk - believable ! meg ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . hard to believe she was the producer on this dog . plus kevin kline : what kind of suicide trip has his career been on ? xxunk ... xxunk ! ! ! finally this was directed by the guy who did big xxunk ? must be a replay of xxunk - hollywood style . xxunk !",negative
"this is a extremely well - made film . the acting , script and camera - work are all first - rate . the music is good , too , though it is mostly early in the film , when things are still relatively xxunk . there are no really xxunk in the cast , though several faces will be familiar . the entire cast does an excellent job with the script . \n\n but it is hard to watch , because there is no good end to a situation like the one presented . it is now xxunk to blame the british for setting hindus and muslims against each other , and then xxunk xxunk them into two countries . there is some merit in this view , but it 's also true that no one forced hindus and muslims in the region to xxunk each other as they did around the time of partition . it seems more likely that the british simply saw the xxunk between the xxunk and were clever enough to exploit them to their own ends . \n\n the result is that there is much cruelty and xxunk in the situation and this is very unpleasant to remember and to see on the screen . but it is never painted as a black - and - white case . there is xxunk and xxunk on both sides , and also the hope for change in the younger generation . \n\n there is redemption of a sort , in the end , when xxunk has to make a hard choice between a man who has ruined her life , but also truly loved her , and her family which has xxunk her , then later come looking for her . but by that point , she has no xxunk that is without great pain for her . \n\n this film carries the message that both muslims and hindus have their grave xxunk , and also that both can be xxunk and caring people . the reality of partition makes that xxunk all the more wrenching , since there can never be real xxunk across the india / pakistan border . in that sense , it is similar to "" mr & mrs xxunk "" . \n\n in the end , we were glad to have seen the film , even though the resolution was xxunk . if the uk and us could deal with their own xxunk of racism with this kind of xxunk , they would certainly be better off .",negative
"every once in a long while a movie will come along that will be so awful that i feel compelled to warn people . if i labor all my days and i can save but one soul from watching this movie , how great will be my joy . \n\n where to begin my discussion of pain . for xxunk , there was a musical montage every five minutes . there was no character development . every character was a stereotype . we had xxunk guy , fat guy who eats donuts , goofy foreign guy , etc . the script felt as if it were being written as the movie was being shot . the production value was so incredibly low that it felt like i was watching a junior high video presentation . have the directors , producers , etc . ever even seen a movie before ? xxunk is getting worse and worse with every new entry . the concept for this movie sounded so funny . how could you go wrong with gary xxunk and a handful of somewhat legitimate actors . but trust me when i say this , things went wrong , xxup very xxup wrong .",negative
"name just says it all . i watched this movie with my dad when it came out and having served in xxunk he had great admiration for the man . the disappointing thing about this film is that it only xxunk on a short period of the man 's life - interestingly enough the man 's entire life would have made such an epic bio - xxunk that it is staggering to imagine the cost for production . \n\n some xxunk xxunk to the flawed xxunk about the man , which are cheap shots . the theme of the movie "" duty , honor , country "" are not just